# Interactive Visual Analytics with Folium 

- mark all launch sites on a map
- mark the success/failed launches for each site on the map
- calculate the distances between a launch site to its proximities

In [1]:
# Import the needed libraries

import requests
import pandas as pd
import io
import folium
from folium import DivIcon
from folium.plugins import MarkerCluster, MousePosition

In [2]:
# The URL below is an augmented dataset with latitude and longitude values added to each site
# We will be using that as our dataset in this task

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
response = requests.get(URL)

# Check if the request was successful
if response.status_code == 200:
    spacex_csv_file = io.BytesIO(response.content)
    spacex_df = pd.read_csv(spacex_csv_file) # "spacex_df" is our dataframe from now on in this task
else:
    print(f"Failed to fetch data. Status code: {response.status_code}")

In [3]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [4]:
# Lets make list for launch site name, latitude and longitude and lets fill those lists
# with the needed values 

nimi_lista = launch_sites_df["Launch Site"].unique().tolist()
lati_lista = launch_sites_df["Lat"].unique().tolist()
longi_lista = launch_sites_df["Long"].unique().tolist()

In [5]:
# Initialize an empty list to store the pairs of 'Lat' and 'Long' values
lat_long_list = []

# Iterate through the rows of the DataFrame
for index, row in launch_sites_df.iterrows():
    lat_long_pair = [row['Lat'], row['Long']]  # Create a pair of 'Lat' and 'Long' values
    lat_long_list.append(lat_long_pair)  # Append the pair to the list

# Now, 'lat_long_list' contains pairs of 'Lat' and 'Long' values for each row
print(lat_long_list)


[[28.56230197, -80.57735648], [28.56319718, -80.57682003], [28.57325457, -80.64689529], [34.63283416, -120.6107455]]


In [6]:
# Lets address NASA Johnson Space Center as our start location:
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

for lat, lon in zip(lati_lista, longi_lista): 
    folium.Marker([lat, lon], tooltip='Click for more info').add_to(site_map)

In [7]:
# Lets create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [8]:
# Test using "while": not efficient

kerroin = 0

while kerroin < len(lat_long_list):

# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
    circle = folium.Circle(lat_long_list[kerroin], radius=1000, color='#d35400', fill=True).add_child(folium.Popup(nimi_lista[kerroin]))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
    marker = folium.map.Marker(
    lat_long_list[kerroin],
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'TESTI',
        )
    )

    kerroin += 1 
    
site_map.add_child(circle)
site_map.add_child(marker)

In [9]:
# Reading data using "iterrows()" works way better than "while"

from folium import DivIcon

# Create a folium Map object
site_map = folium.Map(location=[29.559684888503615, -95.0830971930759], zoom_start=10)

# Create a loop to add circles and markers for each launch site
for index, row in launch_sites_df.iterrows():
    # Get the latitude and longitude values for the current site
    site_location = [row['Lat'], row['Long']]
    
    # Create a circle with a popup label showing the launch site name
    circle = folium.Circle(site_location, radius=1000, color='yellow', fill=True).add_child(folium.Popup(row['Launch Site']))
    
    # Create a marker with a custom icon and popup label showing the launch site name
    marker = folium.Marker(
        location=site_location,
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % row['Launch Site']
        )
    )
    
    # Add both the circle and marker to the map
    site_map.add_child(circle)
    site_map.add_child(marker)

# Display the map
site_map


Lets mark the success/failed launches for each site on the map

In [10]:
värit = [] # Colors

for index, row in spacex_df.iterrows():
    
    classi = row['class']
    if classi == 1:
        värit.append("green")
    if classi == 0:
        värit.append("red")

spacex_df["marker_color"] = värit

spacex_df.head()


,Launch Site,Lat,Long,class,marker_color
0,CCAFS LC-40,28.562302,-80.577356,0,red
1,CCAFS LC-40,28.562302,-80.577356,0,red
2,CCAFS LC-40,28.562302,-80.577356,0,red
3,CCAFS LC-40,28.562302,-80.577356,0,red
4,CCAFS LC-40,28.562302,-80.577356,0,red


In [11]:
# Lets create a folium Map object
site_map = folium.Map(location=[29.559684888503615, -95.0830971930759], zoom_start=10)

# Create a MarkerCluster layer to hold the markers
marker_cluster = MarkerCluster()

# Create a loop to add circles and markers for each launch site
for index, row in launch_sites_df.iterrows():
    # Get the latitude and longitude values for the current site
    site_location = [row['Lat'], row['Long']]
    
    # Create a circle with a popup label showing the launch site name
    circle = folium.Circle(site_location, radius=1000, color='yellow', fill=True).add_child(folium.Popup(row['Launch Site']))
    
    # Create a marker with a custom icon and popup label showing the launch site name
    marker = folium.Marker(
        location=site_location,
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % row['Launch Site']
        )
    )
    
    # Add both the circle and marker to the MarkerCluster
    marker_cluster.add_child(circle)
    marker_cluster.add_child(marker)

# Add the MarkerCluster to the map
site_map.add_child(marker_cluster)

# Display the map
site_map


In [12]:
# Lets create a folium Map object
site_map = folium.Map(location=[29.559684888503615, -95.0830971930759], zoom_start=10)

# Create a MarkerCluster layer to hold the markers
marker_cluster = MarkerCluster()

# Create a loop to add circles and markers for each launch site
for index, row in launch_sites_df.iterrows():
    # Get the latitude and longitude values for the current site
    site_location = [row['Lat'], row['Long']]
    
    # Create a circle with a popup label showing the launch site name
    circle = folium.Circle(site_location, radius=1000, color='yellow', fill=True).add_child(folium.Popup(row['Launch Site']))
    
    # Create a marker with a custom icon and popup label showing the launch site name
    marker = folium.Marker(
        location=site_location,
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % row['Launch Site']
        )
    )
    
    # Add both the circle and marker to the MarkerCluster
    marker_cluster.add_child(circle)
    marker_cluster.add_child(marker)

# Add the MarkerCluster to the map
site_map.add_child(marker_cluster)

# Now, add markers for each launch in spacex_df
for index, record in spacex_df.iterrows():
    # Create a Marker object with its coordinate and customize the Marker's icon property
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],
        icon=folium.Icon(color='white', icon_color=record['marker_color'])  # Customize the icon color
    )
    
    # Add the marker to the MarkerCluster
    marker_cluster.add_child(marker)

# Display the map
site_map

In [13]:
# Lets create a folium Map object
site_map = folium.Map(location=[29.559684888503615, -95.0830971930759], zoom_start=10)

# Create a MarkerCluster layer to hold the markers
marker_cluster = MarkerCluster()

# Create a loop to add circles and markers for each launch site
for index, row in launch_sites_df.iterrows():
    # Get the latitude and longitude values for the current site
    site_location = [row['Lat'], row['Long']]
    
    # Create a circle with a popup label showing the launch site name
    circle = folium.Circle(site_location, radius=1000, color='yellow', fill=True).add_child(folium.Popup(row['Launch Site']))
    
    # Create a marker with a custom icon and popup label showing the launch site name
    marker = folium.Marker(
        location=site_location,
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % row['Launch Site']
        )
    )
    
    # Add both the circle and marker to the MarkerCluster
    marker_cluster.add_child(circle)
    marker_cluster.add_child(marker)

# Add the MarkerCluster to the map
site_map.add_child(marker_cluster)

# Now, add markers for each launch in spacex_df
for index, record in spacex_df.iterrows():
    # Create a Marker object with its coordinate and customize the Marker's icon property
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],
        icon=folium.Icon(color='white', icon_color=record['marker_color'])  # Customize the icon color
    )
    
    # Add the marker to the MarkerCluster
    marker_cluster.add_child(marker)

# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)

# Display the map
site_map


In [14]:
# Lets create a distance calculator

from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

launch_sites_df

for index, row in launch_sites_df.iterrows():
    # Get the latitude and longitude values for the current site
    site_location = [row['Lat'], row['Long']]
    lat1 = site_location[0]
    lon1 = site_location[1]
    lat2 = 28.55946
    lon2 = -80.50713
    print(f"Distance from this point is {calculate_distance(lat1,lon1,lat2,lon2)} km")
    print()

Distance from this point is 6.868000475462204 km

Distance from this point is 6.820963027482625 km

Distance from this point is 13.73949574398502 km

Distance from this point is 3833.5779453862856 km



In [15]:
# Final product

# Lets create a folium Map object
site_map = folium.Map(location=[29.559684888503615, -95.0830971930759], zoom_start=10)

# Create a MarkerCluster layer to hold the markers
marker_cluster = MarkerCluster()

# Create a loop to add circles and markers for each launch site
for index, row in launch_sites_df.iterrows():
    # Get the latitude and longitude values for the current site
    site_location = [row['Lat'], row['Long']]
    
    # Create a circle with a popup label showing the launch site name
    circle = folium.Circle(site_location, radius=1000, color='yellow', fill=True).add_child(folium.Popup(row['Launch Site']))
    
    # Create a marker with a custom icon and popup label showing the launch site name
    marker = folium.Marker(
        location=site_location,
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % row['Launch Site']
        )
    )
    
    # Add both the circle and marker to the MarkerCluster
    marker_cluster.add_child(circle)
    marker_cluster.add_child(marker)

# Add the MarkerCluster to the map
site_map.add_child(marker_cluster)

# Now, add markers for each launch in spacex_df
for index, record in spacex_df.iterrows():
    # Create a Marker object with its coordinate and customize the Marker's icon property
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],
        icon=folium.Icon(color='white', icon_color=record['marker_color'])  # Customize the icon color
    )
    
    # Add the marker to the MarkerCluster
    marker_cluster.add_child(marker)

# Draw a PolyLine between the launch site and selected point (in this case, the west coasts launch control center)
launch_site = [28.573255, -80.646895]  # Launch site coordinates of the launch site KSC-LC 39A
launch_control_center = [28.58523, -80.64915]  # Selected point coordinates
polyline = folium.PolyLine([launch_site, launch_control_center], color='blue', weight=2.5, opacity=1)
site_map.add_child(polyline)

# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)

# Display the map
site_map

Final map product with distance calculator that tells us the distance between launch site to its proximities

In [16]:
# Final product with distance calculator added on

import folium
from folium import DivIcon
from folium.plugins import MarkerCluster, MousePosition

# Create a folium Map object
site_map = folium.Map(location=[29.559684888503615, -95.0830971930759], zoom_start=10)

# Create a MarkerCluster layer to hold the markers
marker_cluster = MarkerCluster()

# Create a loop to add circles and markers for each launch site
for index, row in launch_sites_df.iterrows():
    # Get the latitude and longitude values for the current site
    site_location = [row['Lat'], row['Long']]
    
    # Create a circle with a popup label showing the launch site name
    circle = folium.Circle(site_location, radius=1000, color='yellow', fill=True).add_child(folium.Popup(row['Launch Site']))
    
    # Create a marker with a custom icon and popup label showing the launch site name
    marker = folium.Marker(
        location=site_location,
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % row['Launch Site']
        )
    )
    
    # Add both the circle and marker to the MarkerCluster
    marker_cluster.add_child(circle)
    marker_cluster.add_child(marker)

# Add the MarkerCluster to the map
site_map.add_child(marker_cluster)

# Now, add markers for each launch in spacex_df
for index, record in spacex_df.iterrows():
    # Create a Marker object with its coordinate and customize the Marker's icon property
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],
        icon=folium.Icon(color='white', icon_color=record['marker_color'])  # Customize the icon color
    )
    
    # Add the marker to the MarkerCluster
    marker_cluster.add_child(marker)

# Draw a PolyLine between the launch site and selected point (in this case, the west coasts launch control center)
launch_site = [28.573255, -80.646895]  # Launch site coordinates of the launch site KSC-LC 39A
launch_control_center = [28.58523, -80.64915]  # Selected point coordinates
polyline = folium.PolyLine([launch_site, launch_control_center], color='blue', weight=2.5, opacity=1)
distance_km = calculate_distance(launch_site[0], launch_site[1], launch_control_center[0], launch_control_center[1])
icon = folium.DivIcon(html=f"<div>{distance_km:.2f} km</div>", icon_size=(100, 30))
marker = folium.Marker(location=launch_control_center, icon=icon)
site_map.add_child(marker)
site_map.add_child(polyline)

# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)

# Display the map
site_map